## High dynamic range and environment maps

This example shows how to:
   - setup camera for baking 360 degree panoramic environment map
   - save image file with 8/16/32 bits per sample color depth
   - use OpenCV to save image in HDR format

![plotoptix ray_tracing_output](https://plotoptix.rnd.team/images/360deg_env_map.jpg "This notebook output")

Saving images to jpg format is great when you want to use them quickly, with no additional editing outside your script. Jpg uses 8 bit/sample color depth and lossy compression - such output of your work is ready to go e.g. to the web.

If you plan to apply retouching in an image editing software you'll appreciate saving your renders to a lossless format, with 16 bit/sample color depth. PlotOptiX can do that, tiff (Linux/Windows) and png (Windows) formats are supported.

However, the full information is preserved only if 32 bit/sample, floating point precision data, is saved to a file. Such a high dynamic range (HDR) representation keeps all the bright lights and details in shadows without clamping and rounding errors. Exposure and any tonal corrections can be re-adjusted in HDR images without quality losses. And, most importantly, HDR images are the best for lighting scenes as environment maps - that is what we are going to show in this example. HDR image are written by PlotOptiX natively in tiff format, or you can use OpenCV to save such images in the Radiance file format (.hdr).

In [1]:
import numpy as np
from plotoptix import TkOptiX

Generate some data - balls distributed on a sphere:

In [2]:
n = 2000
r0 = 0.1
R = 4

r = r0 + 0.3 * np.random.rand(n)

x = np.random.normal(loc=0, scale=1.0, size=n)
y = np.random.normal(loc=0, scale=1.0, size=n)
z = np.random.normal(loc=0, scale=1.0, size=n)
xyz = np.stack((x, y, z)).T
for i in range(n): xyz[i] *= R / np.linalg.norm(xyz[i])

A simple function to signal that ray tracing has finished:

In [3]:
def accum_done(rt: TkOptiX) -> None:
    print("rt completed!")

Setup the ray tracing parameters. **Note** that AI denoiser is NOT applied. It could result with a visible seam at the line joining vertical edges of the image, when the image is displayed as an environment map. Instead, only a gamma correction is used and you need to accumulate enough data to reduce the noise.

In [4]:
rt = TkOptiX(on_rt_accum_done=accum_done)
rt.set_param(
    min_accumulation_step=2,
    max_accumulation_frames=300
)
rt.set_uint("path_seg_range", 4, 8) # a little more than the default (2,6) to improve the ambient occlusion impression

exposure = 1; gamma = 1.7
rt.set_float("tonemap_exposure", exposure)
rt.set_float("tonemap_gamma", gamma)
rt.add_postproc("Gamma")

Setup lighting: one bright warm spherical light and some cold light from the ambient.

In [5]:
rt.setup_light("l1", pos=[1.5, 0, 1.5], color=[3.5, 3.2, 2.8], radius=0.75, in_geometry=False)

rt.set_ambient([0.1, 0.2, 0.3])
rt.set_background(0)

Setup cameras: one for making the panoramic view, one to show balls from inside the sphere, and one looking from a distance.

In [6]:
rt.setup_camera("cam1", cam_type="Panoramic", eye=[0, 0, 0], target=[0, 0, -1], up=[0, 1, 0])

rt.setup_camera("cam2", cam_type="DoF",
                eye=[0, 0, 2], target=[0, 0, 0], up=[0, 1, 0],
                aperture_radius=0.2, fov=45, focal_scale=2.8)

rt.setup_camera("cam3", cam_type="DoF",
                eye=[0, 0, 10], target=[0, 0, 0], up=[0, 1, 0],
                aperture_radius=0.07, fov=35, focal_scale=0.56)

Upload data points:

In [7]:
rt.set_data("points", pos=xyz, r=r, c=0.7)

Open the GUI window:

In [8]:
rt.show()

Switch camera views. Let the ray tracing to converge with *cam1* active, this is the image to be used in the next example.

In [9]:
rt.set_current_camera("cam2")

In [10]:
rt.set_current_camera("cam1")

Save images with 8, 16 and 32 bit/sample color depths. 360 degree environment maps can be inspected with the script ``7_panorama_viewer.py`` or used by the notebook ``10_2_read_hdr_360deg_env_map.ipynb``.

**Note:** wait until the image appears in the GUI window; before that image buffers are empty. See callback examples (e.g. [this](https://github.com/rnd-team-dev/plotoptix/blob/master/examples/2_animations_and_callbacks/0_wait_for_raytracing_done.py) or [this](https://github.com/rnd-team-dev/plotoptix/blob/master/examples/2_animations_and_callbacks/0_wait_for_raytracing_done.py)) on how to wait for the result in the code. Here, only a simple message is printed when accumulation is done (see cell #3).

In [11]:
rt.save_image("rt_output_8bps.jpg")

In [12]:
rt.save_image("rt_output_16bps.tif", bps="Bps16")

In [13]:
rt.save_image("rt_output_32bps.tif", bps="Bps32")

Save the image also in Radiance file, if you have OpenCV installed.

In [14]:
import cv2

a = rt.get_rt_output("Bps32", "BGR") # ray tracing output in 32bps depth and channels order required for .hdr format
print(a.dtype, a.shape, np.max(a))   # note it is a floating point array, and strong lights are above 1.0 values

cv2.imwrite('rt_output_32bps.hdr', a)

float32 (1084, 1924, 3) 1.1936004


True

In [15]:
rt.close()